**Setup dell'ambiente**

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

Per questo compito mi occupo di importare il file loans_lenders, di analizzarne la struttura e successivamente decidere come procedere.

In [3]:
loans_lenders = pd.read_csv(loans_lenders_path)

In [4]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Vedendo la struttura del dataframe, la normalizzazione della tabella consiste nel separare gli elementi presenti nella colonna lenders in modo che il dataframe sia formato da celle in cui è presente solo una istanza. Per farlo separo gli elementi che si trovano in corrispondenza della virgola, essendoci uno spazio vuoto dopo la virgola dovrò stare attento a considerarlo. Per ovviare a questo problema trasformo gli elementi ad ogni istanza di lenders in una lista.

In [5]:
loans_lenders['lenders'] = loans_lenders.apply(lambda row: row['lenders'].split(','), axis = 1)

Per separare gli elementi utilizzo la funzione *explode* implementata all'interno della libreria pandas che permette di sepoarare gli elementi di una lista lungo l'asse voluto.

In [6]:
loans_lenders_norm = loans_lenders.explode('lenders')

In [7]:
# Decommentare questa riga per vedere i risultati.
loans_lenders_norm.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


**2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.**

Come per il punto precedente procedo con una esplorazione preliminare del file in cui sono presenti gli attributi utili allo svolgimento del compito. Dopo averne fatto una prima visualizzazione degli attributi si nota che è utile passare gli attributi "disburse_time" e "planned_expiration_time" come date al momento dell'importazione.

In [8]:
loans = pd.read_csv(loans_path, parse_dates=["disburse_time", "planned_expiration_time"])

In [9]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [10]:
loans["duration"] = loans["planned_expiration_time"] - loans["disburse_time"]

Controllo per sicurezza quanti sono gli attributi nulli una volta fatta la differenza. In particolare è un numero abbastanza grande.

In [11]:
round(len(loans.loc[loans['duration'].isnull() == True])/len(loans)*100,2)

26.39

**3) Find the lenders that have funded at least twice.**

Per fare questa cosa ho gli attributi già pronti. Per prima cosa bisogna effettuare un merge tra loans_lenders_norm e loans per sapere se tutte le istanze presenti in loans_lenders norm corrispondano allo stato "funded", una volta fatto questo mi occupo di tenere solo le righe in cui lo stato è funded e di selezionare solo quelle con un conteggio maggiore di o uguale a 2. Per non sovraccaricare troppo i conti mi preoccupo di fare il merge solo con gli attributi che serviranno durante i conti.

In [12]:
loans_lenders_merged = pd.merge(loans_lenders_norm, loans[["loan_id", "status"]], on="loan_id", how="left")

In [13]:
loans_lenders_merged_funded = loans_lenders_merged[loans_lenders_merged["status"] == "funded"]

In [14]:
num_loan = loans_lenders_merged_funded.groupby('lenders').count()['loan_id']
num_loan.loc[num_loan >= 2]

lenders
 000               39
 00000             38
 0002              70
 0101craign0101    71
 0132575            4
                   ..
zyrah8525           4
zyrorl              3
zzaman              9
zzanita             2
zzmcfate           54
Name: loan_id, Length: 993782, dtype: int64

**4) For each country, compute how many loans have involved that country as borrowers.**

Durante l'esecuzione di questo punto nasce la problematica causata dal gran numero di NaN presenti nell'attributo duration notato durante l'esecuzione del punto 2. Per effettuare un calcolo corretto bisogna quindi mettere delle condizioni, in particolare creo un nuovo dataframe che contenga le righe di loans con le seguenti caratteristiche:

- Non hanno valori null in "duration".
- Per quanto riguarda i valori in "planned_expiration_time" che siano minori di "disburse_time" vengono cancellati visto che sono pochi (circa 1\% come mostrato sotto). 

In [15]:
loans_not_null = loans[loans["duration"].isnull() == False]
condition = loans_not_null[loans_not_null["planned_expiration_time"] < loans_not_null["disburse_time"]]
round(len(condition)/len(loans_not_null)*100,2)

1.43

In [16]:
loans_not_null = loans_not_null[loans_not_null["planned_expiration_time"] > loans_not_null["disburse_time"]]

Ora mi occuppo di raggruppare per country e di contare solo quelli corretti.

In [17]:
num_loans_country = loans_not_null.groupby('country_name').count()['loan_id']

In [18]:
#Decommentare questa riga per la visualizzazione dei risultati.
num_loans_country.head()

country_name
Afghanistan        1
Albania         3075
Armenia        12809
Azerbaijan      4026
Belize            98
Name: loan_id, dtype: int64

**5) For each country, compute the overall amount of money borrowed.** 

Per farlo è gia a disposizione il dataset sistemato, bisogna solo raggruppare e sommare per paese.

In [19]:
borr_country = loans_not_null.groupby('country_name')['loan_amount'].sum()
borr_country = borr_country.reset_index().rename(columns={"loan_amount" : "borrowed_amount"})

In [20]:
#Decommentare questa riga per vedere i risultati
#borr_country

**6) Like the previous point, but expressed as a percentage of the overall amount lent.**

Allo stesso modo del punto precedente, basta stare attenti a fare il conto giusto.

In [21]:
overall_amount_lent = loans_not_null['loan_amount'].sum()
borr_country_perc = pd.DataFrame(loans_not_null.groupby('country_name')['loan_amount'].sum())
borr_country_perc['percentage'] = round(borr_country_perc['loan_amount'] / overall_amount_lent * 100, 2)
borr_country_perc = borr_country_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

In [22]:
#Decommentare questa riga per vedere i risultati.
borr_country_perc.head()

,loan_amount_perc,percentage
country_name,,
Afghanistan,6000.0,0.00
Albania,4307350.0,0.50
Armenia,20579275.0,2.37
Azerbaijan,6839400.0,0.79
Belize,65450.0,0.01


Eseguo la somma per vedere che il risultato è unitario, ciò è vero a meno di approssimazioni che sono state svolte durante i conteggi:

In [23]:
round(borr_country_perc['percentage'].sum(),2)

99.98

**7) Like the three previous points, but split for each year (with respect to disburse time).**

Il primo passaggio è quello di riprendere il dataset creato in precedenza e settare l'attributo "disburse_time" come indice in modo da poterci applicare il metodo Grouper di pandas.

In [24]:
loans_not_null["disburse_time"] =  pd.to_datetime(loans_not_null["disburse_time"])
loans_not_null = loans_not_null.set_index("disburse_time")
loans_by_year_sum = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_amount"].sum().to_frame()

Ora faccio le stesse cose fatte sopra ma calcolando la percentuale. Per farlo devo contare quanti sono stati e dividere il count per il "loan_amount" per il conteggio.

In [25]:
loans_by_year_perc = loans_by_year_sum
loans_by_year_perc["loan_amount"] = loans_by_year_perc["loan_amount"]/overall_amount_lent*100
loans_by_year_perc = loans_by_year_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

Faccio come sopra in modo da poter contare questa volta. Rinominiamo sempre gli attributi in modo da permettere una maggiore leggibilità.

In [26]:
loans_by_year_count = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_id"].count().to_frame()
loans_by_year_count = loans_by_year_count.rename(columns={"loan_id" : "count"})

Eseguo il merge tra i tre dataframe selezionando solo le colonne che non sono duplicate in modo da avere il minor spreco di memoria possibile.

In [27]:
country_statistics_by_year = pd.concat([loans_by_year_count, loans_by_year_sum, loans_by_year_perc], axis=1)
country_statistics_by_year = country_statistics_by_year.loc[:, ~country_statistics_by_year.columns.duplicated()]
country_statistics_by_year = country_statistics_by_year.drop("loan_amount", axis = 1)

In [28]:
#Decommentare questa riga per stampare i risultati.
country_statistics_by_year.head()

count  loan_amount_perc
country_name disburse_time                                     
Afghanistan  2015-12-31 00:00:00+00:00      1          0.000692
Albania      2012-12-31 00:00:00+00:00    332          0.047604
             2013-12-31 00:00:00+00:00    507          0.090706
             2014-12-31 00:00:00+00:00    603          0.104031
             2015-12-31 00:00:00+00:00    638          0.097325

**8) For each lender, compute the overall amount of money lent.**

La prima cosa da fare è capire per ogni id il numero di lenders coinvolti, e ciò viene fatto contando il numero di persone presenti nel loan_id. L'assunzione fatta è che ogni lenders si sia occupato in egual misura di ogni loan. Per farlo uso il dataset già normalizzato creato in precedenza.

In [29]:
lenders_num = loans_lenders_norm.groupby("loan_id").count()
lenders_num = lenders_num.reset_index().rename(columns={"lenders" : "lenders_count"})

In [30]:
lenders_num_details = pd.merge(lenders_num, loans_not_null, on="loan_id")[["loan_id", "lenders_count", "loan_amount"]]
lenders_num_details["amount_per_person"] = lenders_num_details["loan_amount"] / lenders_num_details["lenders_count"]

In [31]:
loans_lenders_merged = pd.merge(loans_lenders_norm, lenders_num_details, on="loan_id", how="left")

In [32]:
lenders_overall_lent = loans_lenders_merged.groupby("lenders")["amount_per_person"].sum()
lenders_overall_lent = lenders_overall_lent.to_frame().reset_index()
lenders_overall_lent['amount_per_person'] = round(lenders_overall_lent['amount_per_person'],2)

In [33]:
#Decommentare questa riga per vedere i risultati
lenders_overall_lent.head()

,lenders,amount_per_person
0,000,1283.36
1,00000,0.00
2,0002,677.08
3,00mike00,52.63
4,0101craign0101,580.11


**9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.**

La prima cosa da fare è sistemare il dataframe in modo da averlo senza valori NaN. Una volta diviso il dataframe in queste due parti cerco di calcolare la distribuzione nel dataframe in cui tutto è conosciuto e la applico al secondo dataframe.

In [34]:
lenders = pd.read_csv(lenders_path)

In [35]:
lenders_notnull = lenders.loc[lenders["country_code"].isnull() == False].reset_index()   
lenders_null = lenders.loc[lenders["country_code"].isnull() == True].reset_index()   

Guardo qual è la percentuale dei not null. Come notiamo il numero di valori nulli è molto alto per questo necessita di un buon riempimento.

In [36]:
round(len(lenders_null) / len(lenders)*100, 2)

62.09

Calcolo la distribuzione delle nazioni nel dataframe lenders_notnull

In [37]:
country_ripartition = lenders_notnull[["index", "country_code"]].groupby("country_code").count()
country_ripartition = country_ripartition.reset_index().rename(columns = {"index":"n_users"})
country_ripartition["percentage"] = country_ripartition["n_users"]/len(lenders_notnull)*100

Ora riempio il dataframe lenders_null in modo che abbia la stessa distribuzione di lenders_notnull.
Per fare questo uso la funzione np.random.choice (inserendo come seed '7') in modo che i risultati a cui sono giunto siano riproducibili.
In questo modo assegno la nazionalità in modo randomico, ottenendo la distribuzione finale che voglio ottenere per fare questo devo però normalizzare le percentuali.

In [38]:
country_ripartition["percentage"] /= country_ripartition["percentage"].sum()

In [39]:
np.random.seed(7)
lenders_null["country_code"] = np.random.choice(country_ripartition["country_code"], size=len(lenders_null.index), p = country_ripartition["percentage"])

In [40]:
lenders = pd.concat([lenders_notnull, lenders_null]).drop(columns="index")

Ora che il dataframe è sistemato con la distribuzione giusta, proseguo con la richiesta dell'esercizio.
Eseguo il merge tra il dataframe "lenders" in cui è indicato l'attributo dei lenders e della loro nazionalità e il dataframe "lenders_overall_lent" che indica per ogni lender l'importo totale. A quel punto mi occupo di effettuare la giusta query. Come al solito per appesantire meno i conti cancello gli attributi che non sono necessari.

In [41]:
tot_lent_country = pd.merge(lenders_overall_lent, lenders[["permanent_name", "country_code"]], left_on="lenders", right_on="permanent_name")
tot_lent_country = tot_lent_country.drop(columns="permanent_name")
tot_lent_country = tot_lent_country.groupby("country_code")["amount_per_person"].sum()
tot_lent_country = tot_lent_country.to_frame().rename(columns={"amount_per_person" : "lent_amount"}).reset_index()

Nel dataframe "tot_lent_country" ho indicato il country_code, mentre nel dataframe "tot_borr_country" ho il nome della nazione, informazioni che mi servono entrambe. Eseguo quindi un merge per averle entrambe a portata di mano tenendo solo gli attributi che mi interessano.

In [42]:
country_stats = pd.read_csv(country_stats_path)
loans_country = pd.merge(borr_country, country_stats[["country_name", "country_code"]], on="country_name")

Unisco ora le informazioni riguardo ai money_lent e ai money_borrowed in modo da poter eseguire la differenza tra questi due valori.

In [43]:
country_lent_borr = pd.merge(tot_lent_country, loans_country, on="country_code")
country_lent_borr = country_lent_borr[["country_name", "country_code", "lent_amount", "borrowed_amount"]]
country_lent_borr["difference"] = (country_lent_borr["lent_amount"] - country_lent_borr["borrowed_amount"])

In [44]:
#Decommentare questa riga per vedere i risultati
country_lent_borr.head()

,country_name,country_code,lent_amount,borrowed_amount,difference
0,Afghanistan,AF,6126.40,6000.0,126.40
1,Albania,AL,1495.80,4307350.0,-4305854.20
2,Armenia,AM,1897.69,20579275.0,-20577377.31
3,Azerbaijan,AZ,213.52,6839400.0,-6839186.48
4,Burkina Faso,BF,71.34,3989650.0,-3989578.66


**10) Which country has the highest ratio between the difference computed at the previous point and the population?**

Per risolvere questo punto ho già tutte le informazioni che mi servono a portata di mano, le integro facendo un merge e poi svolgo una semplice query. In questo caso il segno delle differenze è importante perché indica se un paese ha finanziato più o meno soldi di quelli che ha ricevuto.

In [45]:
country_lent_borr  = pd.merge(country_lent_borr, country_stats[["country_code", "population"]], on="country_code")
country_lent_borr["ratio"] = country_lent_borr["difference"]/country_lent_borr["population"]
max = country_lent_borr["ratio"].idxmax()
country_max = country_lent_borr.loc[max]

In [46]:
#Decommentare questa riga per vedere i risultati
country_max

country_name       United States
country_code                  US
lent_amount          3.94815e+07
borrowed_amount      2.82160e+07
difference           1.12654e+07
population             324459463
ratio                  0.0347206
Name: 67, dtype: object

**11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?**

Cerco per prima cosa quali sono i paesi sopra la soglia della povertà. Avendo solo pochi valori nulli (circa il 13\%), per non incasinare troppo il calcolo decido di levare tutti i valori nulli. Calcolo infine il ratio sopra la soglia di povertà e lo inserisco nel dataframe. Attraverso il metodo idxmax trovo il massimo.

In [67]:
country_stats_not_null = country_stats[country_stats["population_below_poverty_line"].isnull() == False]
round(len(country_stats_not_null)/len(country_stats)*100,2)

87.36

L'unico modo che ho per n

In [71]:
pd.options.mode.chained_assignment = None 
country_stats_not_null["population_above_poverty_line"] = (country_stats_not_null["population"] -(country_stats_not_null["population"]*country_stats_not_null["population_below_poverty_line"]/100))

Come al solito durante il merge tengo solo gli attributi significativi.

In [72]:
country_lent_borr  = pd.merge(country_lent_borr, country_stats_not_null[["country_code", "population_above_poverty_line"]], on="country_code")

In [73]:
country_lent_borr["ratio_above_poverty"] = country_lent_borr["difference"]/country_lent_borr["population_above_poverty_line"]
max = country_lent_borr["ratio_above_poverty"].idxmax()
country_lent_borr.loc[max]

country_name                       United States
country_code                                  US
lent_amount                          3.94815e+07
borrowed_amount                      2.82160e+07
difference                           1.12654e+07
population                             324459463
ratio                                  0.0347206
population_above_poverty_line_x      2.75466e+08
ratio_above_poverty                    0.0408959
population_above_poverty_line_y      2.75466e+08
population_above_poverty_line        2.75466e+08
Name: 64, dtype: object

**12)  For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.**

La prima cosa da fare per velocizzare i conti è suddividere il dataframe in due dataframe, il primo conterrà solo gli elementi per cui "disburse year" e "expiration year" si collocano nello stesso anno, il secondo in cui gli anni dei due attributi sono differenti. Questa suddivisione nasce dal fatto che il conto per gli elementi che si trovano nello stesso annoè notevolmente più facile.

Per prima cosa resetto gli indici del dataframe che ho già pronto e mi occupo di rimuovere il fuso orario e le ore poiché sono ininfluenti nel calcolo

In [51]:
loans_not_null = loans_not_null.reset_index()

loans_not_null["disburse_time"] = loans_not_null["disburse_time"].dt.tz_localize(None)
loans_not_null["planned_expiration_time"] = loans_not_null["planned_expiration_time"].dt.tz_localize(None)
loans_not_null["disburse_time"] = loans_not_null["disburse_time"].dt.normalize()
loans_not_null["planned_expiration_time"] = loans_not_null["planned_expiration_time"].dt.normalize()

Divido ora il dataframe nei due sottoinsiemi

In [52]:
loans_same_year = loans_not_null[loans_not_null["disburse_time"].dt.year == loans_not_null["planned_expiration_time"].dt.year][["loan_id" , "disburse_time", "planned_expiration_time","loan_amount"]]
loans_diff_year = loans_not_null[loans_not_null["disburse_time"].dt.year != loans_not_null["planned_expiration_time"].dt.year][["loan_id" ,"disburse_time", "planned_expiration_time","loan_amount"]]

Mi occupo prima del calcolo di quelli dello stesso anno

In [53]:
loans_same_year_tot = loans_same_year.groupby(loans_same_year["disburse_time"].dt.year)["loan_amount"].sum()
loans_same_year_tot = loans_same_year_tot.to_frame().reset_index()
loans_same_year_tot.columns = ['year', 'amount_same']

In [54]:
#Decommentare la riga seguente per vedere i risultati
loans_same_year_tot

,year,amount_same
0,2012,103911725.0
1,2013,98427750.0
2,2014,120644250.0
3,2015,131208475.0
4,2016,133271575.0
5,2017,144870625.0
6,2018,85300.0


Adesso creo la funzione che calcola il loan amount per ogni anno. Per farlo noto che:

$amount_{year} = \frac{amount_{totale}}{day_{start} + 365 * ({year_{end} - year_{start} - 1}) + day_{end}} * day_{year} = \frac{amount_{totale}}{tot_{days}} * day_{year} $ 

Dove:

- $day_{start}$ indica i giorni che intercorrono tra l'inizio del prestito e la fine del primo anno.
- $day_{end}$ indica i giorni che intercorrono tra l'inizio dell'ultimo anno e la fine del prestito.

Notiamo quindi che per ogni anno il coefficiente davanti a $day_{year}$ è costante e facilmente calcolabile.
Il punto cruciale per avere meno tempo di computazione sta nel lavorare sulle colonne, per farlo devo duplicare le righe in modo da avere tutti gli anni intermedi e successivamente applicare la formula scritta sopra alle colonne. *Devo quindi duplicare ogni riga tante volte quanti sono gli anni intermedi* e ciò equivale a:

${year_{end} - year_{start} - 1}$

Cro per prima cosa le righe duplicate:

In [55]:
loans_diff_year = loans_diff_year.loc[loans_diff_year.index.repeat(loans_diff_year['planned_expiration_time'].dt.year - loans_diff_year['disburse_time'].dt.year + 1)]

Ora creo la colonna 'time_int_start' che tiene memoria dell'inizio dell'anno intermedio, che corrisponderà sempre a 1/1/20xx, successivamente creo la colonna 'time_int_end' che sarà sempre del tipo 31/12/20xx. In questo modo effettuare le differenze sulle colonne sarà molto più veloce che non farlo sulle righe.

In [56]:
y = loans_diff_year["disburse_time"].dt.year

loans_diff_year["time_int_start"] = loans_diff_year.drop_duplicates()["disburse_time"].dt.year.apply(lambda x: datetime.datetime(x, 1, 1)) 
loans_diff_year["time_int_end"] = pd.to_datetime(loans_diff_year.groupby(loans_diff_year["loan_id"]).cumcount() + y, format='%Y')

Ora vado a popolare quei due attributi nel modo che ho descritto sopra.

In [57]:
loans_diff_year["time_int_start"] = pd.to_datetime(loans_diff_year.groupby(loans_diff_year["loan_id"]).cumcount() + y, format='%Y')
loans_diff_year["time_int_end"] = loans_diff_year["time_int_start"].dt.year.apply(lambda x: datetime.datetime(x, 12, 31)) 

Calcolo quindi i $day_{year}$ dell'equazione scritta in precedenza. E' importante resettare l'indice perché potrebbe essersi scombinato dopo le duplicazioni.

In [58]:
loans_diff_year = loans_diff_year.reset_index()

In [59]:
loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year > loans_diff_year["time_int_end"].dt.year) & (loans_diff_year["disburse_time"].dt.year == loans_diff_year["time_int_start"].dt.year), "days_per_year"] = (loans_diff_year["time_int_end"] - loans_diff_year["disburse_time"]).dt.days +1                                      
loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year > loans_diff_year["time_int_end"].dt.year) & (loans_diff_year["disburse_time"].dt.year != loans_diff_year["time_int_start"].dt.year), "days_per_year"] = (loans_diff_year["time_int_end"] - loans_diff_year["time_int_start"]).dt.days +1                                           
loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year == loans_diff_year["time_int_end"].dt.year), "days_per_year"] = (loans_diff_year["planned_expiration_time"] - loans_diff_year["time_int_start"]).dt.days +1    

Stampo per vedere se il dataframe è come voglio.

In [60]:
loans_diff_year.head()

,index,loan_id,disburse_time,planned_expiration_time,loan_amount,time_int_start,time_int_end,days_per_year
0,0,657307,2013-12-22,2014-02-14,125.0,2013-01-01,2013-12-31,10.0
1,0,657307,2013-12-22,2014-02-14,125.0,2014-01-01,2014-12-31,45.0
2,1,657259,2013-12-20,2014-03-26,400.0,2013-01-01,2013-12-31,12.0
3,1,657259,2013-12-20,2014-03-26,400.0,2014-01-01,2014-12-31,85.0
4,4,656933,2013-12-17,2014-02-13,425.0,2013-01-01,2013-12-31,15.0


Calcolo quindi i $tot_{days}$ dell'equazione scritta in precedenza ossia il denominatore.

In [61]:
loans_diff_year["tot_days"] = (loans_diff_year["planned_expiration_time"] - loans_diff_year["disburse_time"]).dt.days +1     

Applico quindi la formula scritta in precedenza e creo il dataframe che dovrò unire a quello calcolato in precedenza:

In [62]:
loans_diff_year["loan_amount_per_year"] = (loans_diff_year["loan_amount"]* loans_diff_year["days_per_year"])/loans_diff_year["tot_days"]
loans_diff_year_tot = loans_diff_year.groupby(loans_diff_year["time_int_start"].dt.year)["loan_amount_per_year"].sum()
loans_diff_year_tot = loans_diff_year_tot.to_frame().reset_index()
loans_diff_year_tot.columns = ['year', 'amount_diff'] 

In [63]:
# Decommentare questa riga per vedere i risultati
loans_diff_year_tot

,year,amount_diff
0,2011,6.614928e+05
1,2012,9.251024e+06
2,2013,2.512557e+07
3,2014,3.001850e+07
4,2015,2.394231e+07
5,2016,2.081552e+07
6,2017,2.046922e+07
7,2018,4.877086e+06


Ora che ho i due dataframe pronti faccio un merge in modo da avere le colonne affiancate e le sommo. I valori nulli che avevo prima devo fare in modo di sostituirli con 0 affinché la somma tra i due mi restituisca un numero e non 0.

In [64]:
amount_per_year = pd.merge(loans_diff_year_tot, loans_same_year_tot, on = 'year', how="outer")

In [65]:
amount_per_year[["amount_diff" , "amount_same"]] = amount_per_year[["amount_diff" , "amount_same"]].fillna(0)
amount_per_year["amount"] = amount_per_year["amount_diff"] + amount_per_year["amount_same"]
amount_per_year = amount_per_year[['year', 'amount']]

In [66]:
#Decommentare questa riga per vedere i risultati.
amount_per_year

,year,amount
0,2011,6.614928e+05
1,2012,1.131627e+08
2,2013,1.235533e+08
3,2014,1.506627e+08
4,2015,1.551508e+08
5,2016,1.540871e+08
6,2017,1.653398e+08
7,2018,4.962386e+06
